<ol>
    <li>Visulaize Nulls</li>
    <li>Deal with nulls
    <li>Eliminate</li>
    <li>Infer</li>
    <li>One Hot encode categorical data</li>
    <li>Join the economic conditions data to the loan data</li>
    <li>Convert zip code to location (encode location)</li>
</ol>

In [4]:
pip install yfinance

  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8500 sha256=cb1bd80425e7dbbbef5561b5299f09775d60ef41dd2a06077f3450350c4de613
  Stored in directory: c:\users\ljman\appdata\local\pip\cache\wheels\f2\b5\2c\59ba95dcf854e542944c75fe3da584e4e3833b319735a0546c
Successfully built multitasking
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime


In [20]:
unemployment = pd.read_csv("./data/unemployment2009-2018.csv", index_col=False)
#unemployment.reset_index(drop=True,inplace=True)
unemployment = pd.melt(unemployment, id_vars=['Year'],value_vars=['Jan','Feb','Mar','Apr','May',
                                                                  'Jun','Jul','Aug','Sep','Oct','Nov','Dec'])

unemployment['Date']= unemployment['variable']+ '-'+ unemployment['Year'].astype('str')
unemployment['Date']= pd.to_datetime(unemployment['Date'].astype('str')).dt.strftime('%Y-%m')
unemployment = unemployment.sort_values(by=['Date'])
unemployment = unemployment.reset_index()
unemployment = unemployment.drop(['index'],axis=1)
unemployment.rename(columns={'value':'U_Rate'},inplace=True)
unemployment.drop(['variable','Year'], axis=1,inplace=True)
unemployment.head()

,U_Rate,Date
0,7.8,2009-01
1,8.3,2009-02
2,8.7,2009-03
3,9.0,2009-04
4,9.4,2009-05


In [21]:
import yfinance as yf

spy = yf.download('SPY', start='2009-01-01', end = '2017-12-31')
spy.reset_index(inplace=True)
spy.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-01-02,90.440002,93.440002,89.849998,92.959999,71.841019,227566300
1,2009-01-05,92.629997,93.660004,91.889999,92.849998,71.756012,240349700
2,2009-01-06,93.639999,94.449997,92.680000,93.470001,72.235176,328260900
3,2009-01-07,92.000000,92.260002,90.199997,90.669998,70.071274,280899200
4,2009-01-08,90.160004,91.089996,89.669998,91.040001,70.357193,263834400


In [22]:
df = pd.read_csv("./data/SSBCI Transactions Dataset.csv")
df['disbursement_date']=pd.to_datetime(df['disbursement_date'])
df = df.merge(spy, left_on='disbursement_date', right_on='Date', how = 'left')
df.drop(['Open','High','Low','Adj Close','Volume'], axis=1, inplace=True)
df.shape

(21962, 51)

In [23]:
df.head()

,state_id,state_name,year_reported,program_name,program_type,unique_id,disbursement_date,loan_investment_amount,ssbci_original_funds,nonprivate_amount,...,optional_active,optional_active_no,optional_active_unknown,optional_dollars_lost,optional_business_city,optional_business_state,optional_coinvestment_source,optional_stage,Date,Close
0,AK,Alaska - Anchorage,2013,49th State Venture Fund,Venture Capital,AK_2013_VCP_1,2013-03-01,180000.0,90000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-03-01,152.110001
1,AK,Alaska - Anchorage,2013,49th State Venture Fund,Venture Capital,AK_2013_VCP_2,2013-07-13,2400000.0,250000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,AK,Alaska - Anchorage,2013,49th State Venture Fund,Venture Capital,AK_2013_VCP_3,2013-12-30,265000.0,110000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-12-30,183.820007
3,AK,Alaska - Anchorage,2014,49th State Venture Fund,Venture Capital,AK_2014_VCP_1,2014-04-11,199500.0,99750.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-04-11,181.509995
4,AK,Alaska - Anchorage,2014,49th State Venture Fund,Venture Capital,AK_2014_VCP_2,2014-05-15,125000.0,62500.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-05-15,187.399994


In [24]:
from datetime import timedelta
for i in range(len(df)):
    checkdate=df.loc[i,'disbursement_date']
    #print(checkdate)
    while (~(spy.isin([checkdate]).any().any())):
        checkdate=checkdate-timedelta(days=1)
        if (spy.isin([checkdate]).any().any()):
            df.loc[i,'Close']= spy.loc[spy['Date']==checkdate, 'Close'].item()
        


In [25]:
df.head()

,state_id,state_name,year_reported,program_name,program_type,unique_id,disbursement_date,loan_investment_amount,ssbci_original_funds,nonprivate_amount,...,optional_active,optional_active_no,optional_active_unknown,optional_dollars_lost,optional_business_city,optional_business_state,optional_coinvestment_source,optional_stage,Date,Close
0,AK,Alaska - Anchorage,2013,49th State Venture Fund,Venture Capital,AK_2013_VCP_1,2013-03-01,180000.0,90000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-03-01,152.110001
1,AK,Alaska - Anchorage,2013,49th State Venture Fund,Venture Capital,AK_2013_VCP_2,2013-07-13,2400000.0,250000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,167.509995
2,AK,Alaska - Anchorage,2013,49th State Venture Fund,Venture Capital,AK_2013_VCP_3,2013-12-30,265000.0,110000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-12-30,183.820007
3,AK,Alaska - Anchorage,2014,49th State Venture Fund,Venture Capital,AK_2014_VCP_1,2014-04-11,199500.0,99750.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-04-11,181.509995
4,AK,Alaska - Anchorage,2014,49th State Venture Fund,Venture Capital,AK_2014_VCP_2,2014-05-15,125000.0,62500.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-05-15,187.399994


In [29]:
df = df.merge(unemployment, left_on=df['disbursement_date'].dt.strftime('%Y-%m'), right_on='Date', how='left')
df.rename(columns={'U_Rate_y':'U_Rate','Close':'SPY_Close'},inplace=True)

C:\Users\ljman\anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [30]:
df.drop(['Date_x','Date_y','U_Rate_x','Date'], axis=1,inplace=True)
df.head()

,state_id,state_name,year_reported,program_name,program_type,unique_id,disbursement_date,loan_investment_amount,ssbci_original_funds,nonprivate_amount,...,optional_active,optional_active_no,optional_active_unknown,optional_dollars_lost,optional_business_city,optional_business_state,optional_coinvestment_source,optional_stage,SPY_Close,U_Rate
0,AK,Alaska - Anchorage,2013,49th State Venture Fund,Venture Capital,AK_2013_VCP_1,2013-03-01,180000.0,90000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.110001,7.5
1,AK,Alaska - Anchorage,2013,49th State Venture Fund,Venture Capital,AK_2013_VCP_2,2013-07-13,2400000.0,250000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.509995,7.3
2,AK,Alaska - Anchorage,2013,49th State Venture Fund,Venture Capital,AK_2013_VCP_3,2013-12-30,265000.0,110000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183.820007,6.7
3,AK,Alaska - Anchorage,2014,49th State Venture Fund,Venture Capital,AK_2014_VCP_1,2014-04-11,199500.0,99750.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,181.509995,6.2
4,AK,Alaska - Anchorage,2014,49th State Venture Fund,Venture Capital,AK_2014_VCP_2,2014-05-15,125000.0,62500.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,187.399994,6.3


In [31]:
df.isna().sum()

state_id                             0
state_name                           0
year_reported                        0
program_name                         0
program_type                         0
unique_id                            0
disbursement_date                    0
loan_investment_amount               0
ssbci_original_funds                 0
nonprivate_amount                    0
concurrent_private_financing         0
borrower_insurance_premium           0
lender_insurance_premium             0
guaranteed_amount                    0
collateral_support                   0
ssbci_recycled_funds                 0
subsequent_private_financing         0
zip_code                             0
metro_type                           0
LMI_type                             0
revenue                              0
full_time_employees                  0
naics_code                           0
year_incorporated                    0
jobs_created                         0
jobs_retained            

In [32]:
drop_cols=df.columns[31:-2]
drop_cols

Index(['MDI_type', 'VC_cat', 'optional_woman_owned', 'optional_minority_owned',
       'optional_veteran_owned', 'optional_FTE', 'optional_FTE_yr_confirmed',
       'optional_primary_use_of_funds', 'optional_revenue',
       'optional_revenue_yr_confirmed', 'optional_active',
       'optional_active_no', 'optional_active_unknown',
       'optional_dollars_lost', 'optional_business_city',
       'optional_business_state', 'optional_coinvestment_source',
       'optional_stage'],
      dtype='object')

In [33]:
ssbci_drop_cols=df.drop(drop_cols, axis=1)
ssbci_drop_cols['job_benefit'] = ssbci_drop_cols['jobs_created'] + ssbci_drop_cols['jobs_retained']
ssbci_drop_cols=ssbci_drop_cols.drop(['state_name','year_reported','disbursement_date','unique_id',
                                      'program_name','lender_name','lender_type',
                                     'lender_type_category','zip_code'], axis=1)
ssbci_data = ssbci_drop_cols.copy()
ssbci_data.loc[ssbci_data['jobs_created'] > 0, 'creation_status'] = 1
ssbci_data.loc[ssbci_data['jobs_created'] == 0, 'creation_status'] = 0
#ssbci_data.drop('job_benefit', axis=1, inplace=True)

In [34]:
ssbci_drop_cols.to_csv("./data/ssbci_catdata.csv")
ssbci_data.head()

,state_id,program_type,loan_investment_amount,ssbci_original_funds,nonprivate_amount,concurrent_private_financing,borrower_insurance_premium,lender_insurance_premium,guaranteed_amount,collateral_support,...,naics_code,year_incorporated,jobs_created,jobs_retained,trans_type,CDFI_type,SPY_Close,U_Rate,job_benefit,creation_status
0,AK,Venture Capital,180000.0,90000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,541511,2013,3.0,1.0,VC,VC,152.110001,7.5,4.0,1.0
1,AK,Venture Capital,2400000.0,250000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,423930,2012,4.0,0.0,VC,VC,167.509995,7.3,4.0,1.0
2,AK,Venture Capital,265000.0,110000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,541511,2013,2.0,5.0,VC,VC,183.820007,6.7,7.0,1.0
3,AK,Venture Capital,199500.0,99750.0,0.0,0.0,0.0,0.0,0.0,0.0,...,336612,2014,8.0,0.0,VC,VC,181.509995,6.2,8.0,1.0
4,AK,Venture Capital,125000.0,62500.0,0.0,125000.0,0.0,0.0,0.0,0.0,...,451110,2007,8.0,0.0,VC,VC,187.399994,6.3,8.0,1.0


In [35]:
ssbci_data.isna().sum()

state_id                        0
program_type                    0
loan_investment_amount          0
ssbci_original_funds            0
nonprivate_amount               0
concurrent_private_financing    0
borrower_insurance_premium      0
lender_insurance_premium        0
guaranteed_amount               0
collateral_support              0
ssbci_recycled_funds            0
subsequent_private_financing    0
metro_type                      0
LMI_type                        0
revenue                         0
full_time_employees             0
naics_code                      0
year_incorporated               0
jobs_created                    0
jobs_retained                   0
trans_type                      0
CDFI_type                       0
SPY_Close                       0
U_Rate                          0
job_benefit                     0
creation_status                 0
dtype: int64

In [36]:
ssbci_data.dtypes

state_id                         object
program_type                     object
loan_investment_amount          float64
ssbci_original_funds            float64
nonprivate_amount               float64
concurrent_private_financing    float64
borrower_insurance_premium      float64
lender_insurance_premium        float64
guaranteed_amount               float64
collateral_support              float64
ssbci_recycled_funds            float64
subsequent_private_financing    float64
metro_type                       object
LMI_type                         object
revenue                         float64
full_time_employees             float64
naics_code                        int64
year_incorporated                 int64
jobs_created                    float64
jobs_retained                   float64
trans_type                       object
CDFI_type                        object
SPY_Close                       float64
U_Rate                          float64
job_benefit                     float64


In [37]:
ssbci_obj_cols = ssbci_drop_cols.select_dtypes(exclude=[np.float64]).columns
ssbci_int_cols = ['naics_code','year_incorporated']
#ssbci_obj_cols.append(ssbci_int_cols)
ssbci_obj_cols

Index(['state_id', 'program_type', 'metro_type', 'LMI_type', 'naics_code',
       'year_incorporated', 'trans_type', 'CDFI_type'],
      dtype='object')

In [38]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for i in ssbci_obj_cols:
    ssbci_data[i] = label_encoder.fit_transform(ssbci_drop_cols[i])

print("Label Encoded Data: ")
ssbci_data.head()

Label Encoded Data: 


,state_id,program_type,loan_investment_amount,ssbci_original_funds,nonprivate_amount,concurrent_private_financing,borrower_insurance_premium,lender_insurance_premium,guaranteed_amount,collateral_support,...,naics_code,year_incorporated,jobs_created,jobs_retained,trans_type,CDFI_type,SPY_Close,U_Rate,job_benefit,creation_status
0,0,4,180000.0,90000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,681,103,3.0,1.0,1,2,152.110001,7.5,4.0,1.0
1,0,4,2400000.0,250000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,439,102,4.0,0.0,1,2,167.509995,7.3,4.0,1.0
2,0,4,265000.0,110000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,681,103,2.0,5.0,1,2,183.820007,6.7,7.0,1.0
3,0,4,199500.0,99750.0,0.0,0.0,0.0,0.0,0.0,0.0,...,378,104,8.0,0.0,1,2,181.509995,6.2,8.0,1.0
4,0,4,125000.0,62500.0,0.0,125000.0,0.0,0.0,0.0,0.0,...,519,97,8.0,0.0,1,2,187.399994,6.3,8.0,1.0


In [39]:
ssbci_data.to_csv('./data/SSBCI_data.csv')

In [16]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(ssbci_drop_cols[ssbci_obj_cols])

#Creating a dataframe with encoded data with new column name
onehot_encoded_frame = pd.DataFrame(onehot_encoded, columns = onehot_encoder.get_feature_names(ssbci_obj_cols))
onehot_encoded_frame.head()

,state_id_0,state_id_1,state_id_2,state_id_3,state_id_4,state_id_5,state_id_6,state_id_7,state_id_8,state_id_9,...,disbursement_date_1603,disbursement_date_1604,disbursement_date_1605,disbursement_date_1606,metro_type_0,metro_type_1,LMI_type_0,LMI_type_1,trans_type_0,trans_type_1
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [18]:
onehot_encoded_frame.to_csv('./data/SSBCI_One_Hot.csv') 
